In [37]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.nn.functional import softmax


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erxcshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/erxcshi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [41]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [116]:
def sentiment_score(text):
    token = tokenizer.encode(text, return_tensors="pt", padding=True, truncation=True)
    results = model(token)
    poss = int(torch.argmax(results.logits))
    return poss

In [89]:
df = pd.read_csv('Copy of spring2023_applicant_info_and_reader_scores.csv')
df = df.dropna(subset=['Applicant Name'])
df = df.drop(columns = ['Interview - Qualitative', 'Interviewer', 'Interview - Quantitative', 
                   'Potential Next Semester Applicant', 'Composite Score (Application + Interview)', 
                   'Linkedin', 'Cohort', 'Phone', 'Average Interview Score',
                       'Reached out?', 'Reader Score Average', 'Reader 1 Score', 'Reader 2 Score', 
                        'Reader 3 Score', 'Reader 4 Score', 'Email', 'How did you hear about us?', 
                       'Reader 1', 'Reader 2', 'Reader 3', 'Reader 4', 'Group', 'Group', 'Resume',
                       'Total Reader Score Average', 'Total Variance', 'Variance Flag'])
df = df.rename(columns={'Group 1 average' : 'Average Score', 'Group 1 Variance' : 'Score Variance'})
df = df.drop_duplicates(subset=['Applicant Name'])


In [90]:
df1 = pd.read_excel('Spring 2023 (responses).xlsx')
df1 = df1.rename(columns = {'Name' : 'Applicant Name'})
df1 = df1.dropna(subset = ['Applicant Name'])
df1 = df1[df1['Current Year'] != 'Graduate Student']
df1 = df1[df1['Applicant Name'].isin(df['Applicant Name'])]
df1 = df1.drop_duplicates(subset=['Applicant Name'])
df1 = df1.drop(columns = ['Timestamp', 'Email Address', 'Score', 'Phone (XXX-XXX-XXXX)'])


In [91]:
df = df[df['Applicant Name'].isin(df1['Applicant Name'])]
merged_df = df.merge(df1, how='inner', on='Applicant Name')
merged_df['Average Score'] = merged_df['Average Score'].astype(float).round(2)
merged_df['Range'] = 0
merged_df = merged_df.reset_index(drop=True)
merged_df.columns = [col.strip() for col in merged_df.columns]

In [92]:
ok = []
lower = np.percentile(merged_df['Average Score'], 40)
medium = np.percentile(merged_df['Average Score'], 70)
high = np.percentile(merged_df['Average Score'], 85)

In [93]:
for index, score in enumerate(merged_df['Average Score']):
    if score <= lower:
        merged_df['Range'][index] = 'low'
    elif lower < score <= medium:
        merged_df['Range'][index] = 'medium'
    elif medium < score <= high:
        merged_df['Range'][index] = 'high'
    elif score >= high:
        merged_df['Range'][index] = 'highest'

/var/folders/tb/k3yxh4hd0yd7d43rn3_4f3k40000gn/T/ipykernel_26111/53045529.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Range'][index] = 'low'


In [94]:
stop_words = list(stopwords.words('english'))

punctuation_list = string.punctuation
punctuation_list = list(punctuation_list)

In [95]:
merged_df['Interview?'] = merged_df['Interview?'].fillna(0)
for idx, val in enumerate(merged_df['Interview?']):
    if val == 'Yes':
        merged_df.at[idx, 'Interview?'] = 1
    if val == 'No':
        merged_df.at[idx, 'Interview?'] = 0

In [123]:
response_columns_cf = [
    '1. Why are you interested in joining TAMID, and what do you hope to gain from your experience?',
    '2. At TAMID, we prioritize our tight-knit community! How do you imagine yourself contributing to this community?',
    '3. What skills do you hope to build or improve through your involvement in TAMID?',
    '4. Pick an Israeli company that inspires you, and tell us one thing they could improve upon?',
    "5. Do you have any previous experience in consulting, finance, software development, or any other experiences you believe can transfer to TAMID? (It's okay if it's not a professional experience)",
    '6. Tell us about another community you are a part of. Why do you choose to be a part of this community?',
    "7. What's something you're passionate about that you'd be happy to teach a friend?",
    "8. What is a fun fact about you?",
    "9. What do you think TAMID stands for? (There is no right answer!)",
    "Thank you for applying! Is there anything else that you'd like to add?",
    "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]",
    'Interview?'
]

responses_cf = merged_df[response_columns_cf]

for idx, vals in enumerate(responses_cf["10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"]):
    if vals == 'Investment Fund':
        responses_cf.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 1
    elif vals == 'Consulting':
        responses_cf.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 2
    elif vals == 'Tech Consulting':
        responses_cf.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 3

responses_cf = responses_cf.fillna('0')
for col in responses_cf:
    responses_cf[col].astype(object)
    
sia = SentimentIntensityAnalyzer()

In [124]:
for column in responses_cf.columns[:-2]:
    for index, response in responses_cf[column].items():
        new_words = []
        for word in response.split(' '):
            new_word = ''.join([char.lower().strip() for char in word if char.isalpha() and char not in punctuation_list])
            new_words.append(new_word)
        new_words = ' '.join([word for word in new_words if word not in stop_words])
        score = sia.polarity_scores(new_words)
        responses_cf.at[index, column] = score['compound']
responses_cf

,"1. Why are you interested in joining TAMID, and what do you hope to gain from your experience?","2. At TAMID, we prioritize our tight-knit community! How do you imagine yourself contributing to this community?",3. What skills do you hope to build or improve through your involvement in TAMID?,"4. Pick an Israeli company that inspires you, and tell us one thing they could improve upon?","5. Do you have any previous experience in consulting, finance, software development, or any other experiences you believe can transfer to TAMID? (It's okay if it's not a professional experience)",6. Tell us about another community you are a part of. Why do you choose to be a part of this community?,7. What's something you're passionate about that you'd be happy to teach a friend?,8. What is a fun fact about you?,9. What do you think TAMID stands for? (There is no right answer!),Thank you for applying! Is there anything else that you'd like to add?,"10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]",Interview?
0,0.8271,0.9442,0.5994,0.6808,0.25,0.9382,0.9879,0.0,0.836,0.0,3,0
1,0.9441,0.9201,0.891,0.9423,0.9485,0.886,0.8999,0.0,0.3818,0.0,2,0
2,0.9786,0.9652,0.9726,0.9393,0.6486,0.9559,0.7906,0.802,0.0,0.9118,3,0
3,0.9983,0.9952,0.9936,0.9935,0.969,0.9844,0.9722,0.0,-0.4939,0.9565,3,1
4,0.998,0.9977,0.9957,0.9964,0.9988,0.9985,0.9981,0.7964,0.2094,0.34,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
182,0.9823,0.9423,0.9792,0.9062,0.9382,0.8454,0.9565,0.0,0.8271,0.0,3,0
183,0.9735,0.9403,0.9818,0.9843,0.9884,-0.2228,0.9538,0.0,0.0,0.4404,1,1
184,0.93,0.9756,0.9349,0.9501,0.9584,0.5994,0.9623,0.4939,0.8591,0.8176,2,1
185,0.9959,0.9864,0.9955,0.9628,0.7096,0.9601,0.8834,0.0,0.0,0.0,3,0


In [125]:
X = responses_cf[response_columns_cf[:-1]]
y = responses_cf['Interview?']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81        27
           1       0.85      0.77      0.81        30

    accuracy                           0.81        57
   macro avg       0.81      0.81      0.81        57
weighted avg       0.81      0.81      0.81        57



In [20]:
rf_classifier.feature_importances_

array([0.13802501, 0.14849959, 0.0793753 , 0.09862254, 0.14443594,
       0.10622255, 0.09579285, 0.07098955, 0.04437826, 0.04143248,
       0.03222593])

In [21]:
response_columns_reg = [
    '1. Why are you interested in joining TAMID, and what do you hope to gain from your experience?',
    '2. At TAMID, we prioritize our tight-knit community! How do you imagine yourself contributing to this community?',
    '3. What skills do you hope to build or improve through your involvement in TAMID?',
    '4. Pick an Israeli company that inspires you, and tell us one thing they could improve upon?',
    "5. Do you have any previous experience in consulting, finance, software development, or any other experiences you believe can transfer to TAMID? (It's okay if it's not a professional experience)",
    '6. Tell us about another community you are a part of. Why do you choose to be a part of this community?',
    "7. What's something you're passionate about that you'd be happy to teach a friend?",
    "8. What is a fun fact about you?",
    "9. What do you think TAMID stands for? (There is no right answer!)",
    "Thank you for applying! Is there anything else that you'd like to add?",
    "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]",
    'Average Score'   
]
responses_reg = merged_df[response_columns_reg]

for idx, vals in enumerate(responses_reg["10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"]):
    if vals == 'Investment Fund':
        responses_reg.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 1
    elif vals == 'Consulting':
        responses_reg.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 2
    elif vals == 'Tech Consulting':
        responses_reg.at[idx, "10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]"] = 3

responses_reg = responses_reg.fillna('0')
for col in responses_reg:
    responses_reg[col].astype(object)
    
sia = SentimentIntensityAnalyzer()

for column in responses_reg.columns[:-2]:
    for index, response in responses_reg[column].items():
        new_words = []
        for word in response.split(' '):
            new_word = ''.join([char.lower().strip() for char in word if char.isalpha() and char not in punctuation_list])
            new_words.append(new_word)
        new_words = ' '.join([word for word in new_words if word not in stop_words])
        score = sia.polarity_scores(new_words)
        responses_reg.at[index, column] = score['compound']
responses_reg

,"1. Why are you interested in joining TAMID, and what do you hope to gain from your experience?","2. At TAMID, we prioritize our tight-knit community! How do you imagine yourself contributing to this community?",3. What skills do you hope to build or improve through your involvement in TAMID?,"4. Pick an Israeli company that inspires you, and tell us one thing they could improve upon?","5. Do you have any previous experience in consulting, finance, software development, or any other experiences you believe can transfer to TAMID? (It's okay if it's not a professional experience)",6. Tell us about another community you are a part of. Why do you choose to be a part of this community?,7. What's something you're passionate about that you'd be happy to teach a friend?,8. What is a fun fact about you?,9. What do you think TAMID stands for? (There is no right answer!),Thank you for applying! Is there anything else that you'd like to add?,"10. At the moment, please rank your top 3 interest levels in the TAMID tracks. (if you're not sure, that's okay!) [First Choice]",Average Score
0,0.8271,0.9442,0.5994,0.6808,0.25,0.9382,0.9879,0.0,0.836,0.0,3,5.17
1,0.9441,0.9201,0.891,0.9423,0.9485,0.886,0.8999,0.0,0.3818,0.0,2,4.33
2,0.9786,0.9652,0.9726,0.9393,0.6486,0.9559,0.7906,0.802,0.0,0.9118,3,4.67
3,0.9983,0.9952,0.9936,0.9935,0.969,0.9844,0.9722,0.0,-0.4939,0.9565,3,8.17
4,0.998,0.9977,0.9957,0.9964,0.9988,0.9985,0.9981,0.7964,0.2094,0.34,2,8.67
...,...,...,...,...,...,...,...,...,...,...,...,...
182,0.9823,0.9423,0.9792,0.9062,0.9382,0.8454,0.9565,0.0,0.8271,0.0,3,6.83
183,0.9735,0.9403,0.9818,0.9843,0.9884,-0.2228,0.9538,0.0,0.0,0.4404,1,7.83
184,0.93,0.9756,0.9349,0.9501,0.9584,0.5994,0.9623,0.4939,0.8591,0.8176,2,7.50
185,0.9959,0.9864,0.9955,0.9628,0.7096,0.9601,0.8834,0.0,0.0,0.0,3,5.67


In [22]:
X = responses_reg[response_columns_reg[:-1]]
y = responses_reg['Average Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

predictions = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.153695405964911


In [23]:
rf_regressor.feature_importances_

array([0.29209591, 0.17756571, 0.10151202, 0.07288565, 0.13406932,
       0.05742335, 0.04926133, 0.02580185, 0.03468495, 0.03007262,
       0.0246273 ])